In [1]:
import tensorflow as tf
from flask import Flask, request
import numpy as np
import mediapipe as mp
import cv2
import base64

In [2]:
model = tf.keras.models.load_model('my_model2.keras')

In [3]:
actions = np.array(['hola', 'jugar', 'a', 'e', 'gracias', 'ir','yo', 'neutral','tu','donde', 'dias', 'adios', 'nombre', 'de nada'])

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    right = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    left = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    return np.concatenate([pose, face, right, left])

In [8]:
def decode_image(img):
    image_data = img.split(",")[1]
    image_bytes = base64.b64decode(image_data)
    image_buffer = np.frombuffer(image_bytes, dtype=np.uint8)
    frame = cv2.imdecode(image_buffer, cv2.IMREAD_COLOR)
    return frame

In [9]:
holistic =  mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [10]:
from flask import jsonify, render_template
from flask_socketio import SocketIO, send

from datetime import datetime


In [11]:
app = Flask(__name__)
socketio = SocketIO(app)
socketio.init_app(app, cors_allowed_origins="*")


frames = []
sequence = []
predictions = []
threshold = 0.5

@socketio.on("connect")
def on_connect():
    print("Cliente conectado")

@app.route("/")
def index():
    return render_template("index.html")

@socketio.on("message")
def on_message(data):   
   
    global sequence
    #print(data["frames"])
    image_base64 = data["frames"]
    #start_time = datetime.now()
    frame = decode_image(image_base64)
    #end_time = datetime.now()
    #print(end_time - start_time)
    
    image, results = mediapipe_detection(frame, holistic)
    keypoints = extract_keypoints(results)
    print(keypoints)
    sequence.append(keypoints)
    
    if len(sequence) == 30:
        #print(sequence[1])
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        prediction = actions[np.argmax(res)]
        socketio.emit("prediction", {"prediction": prediction})
        sequence = []

   
if __name__ == "__main__":
    socketio.run(app, host="localhost", port=5000, allow_unsafe_werkzeug=True)

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2024 20:53:59] "GET /socket.io/?EIO=4&transport=polling&t=O_Wz0Hp HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2024 20:53:59] "POST /socket.io/?EIO=4&transport=polling&t=O_Wz0Mn&sid=cSJyntSsmh0KedHRAAAA HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2024 20:54:00] "GET /socket.io/?EIO=4&transport=polling&t=O_Wz0Mo&sid=cSJyntSsmh0KedHRAAAA HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2024 20:54:00] "GET /socket.io/?EIO=4&transport=polling&t=O_Wz0Rk&sid=cSJyntSsmh0KedHRAAAA HTTP/1.1" 200 -


Cliente conectado
[ 0.39033788  0.38638315 -0.58138949 ...  0.          0.
  0.        ]
[ 0.39033788  0.38638315 -0.58138949 ...  0.          0.
  0.        ]
[ 0.38390154  0.3847622  -0.53794825 ...  0.          0.
  0.        ]
[ 0.37958094  0.38330525 -0.53886354 ...  0.          0.
  0.        ]
[ 0.37993792  0.38185683 -0.545344   ...  0.          0.
  0.        ]
[ 0.38015637  0.37937066 -0.53374034 ...  0.          0.
  0.        ]
[ 0.38030288  0.37908089 -0.5397197  ...  0.          0.
  0.        ]
[ 0.37816721  0.37954494 -0.54091823 ...  0.          0.
  0.        ]
[ 0.37693512  0.38003093 -0.543019   ...  0.          0.
  0.        ]
[ 0.37388542  0.38490263 -0.54266095 ...  0.          0.
  0.        ]
[ 0.37117422  0.38625669 -0.50782377 ...  0.          0.
  0.        ]
[ 0.37142026  0.38637149 -0.5148614  ...  0.          0.
  0.        ]
[ 0.37162653  0.38633722 -0.52503997 ...  0.          0.
  0.        ]
[ 0.37167236  0.38627702 -0.52606475 ...  0.          0.
  

127.0.0.1 - - [03/Jun/2024 20:58:54] "GET /socket.io/?EIO=4&transport=websocket&sid=cSJyntSsmh0KedHRAAAA HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2024 20:58:55] "GET /socket.io/?EIO=4&transport=polling&t=O_W-8Pn HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2024 20:58:55] "POST /socket.io/?EIO=4&transport=polling&t=O_W-8Uk&sid=OQN94qdOQ0GKsqGTAAAC HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2024 20:58:55] "GET /socket.io/?EIO=4&transport=polling&t=O_W-8Ul&sid=OQN94qdOQ0GKsqGTAAAC HTTP/1.1" 200 -


Cliente conectado
[ 0.3831529   0.41573146 -0.61744583 ...  0.          0.
  0.        ]
[ 0.38310963  0.40481272 -0.61135566 ...  0.          0.
  0.        ]
[ 0.38354522  0.40099293 -0.64007527 ...  0.          0.
  0.        ]
[ 0.38357684  0.39933687 -0.60925603 ...  0.          0.
  0.        ]
[ 0.38328716  0.39822811 -0.61362398 ...  0.          0.
  0.        ]
[ 0.38302743  0.39851871 -0.59882587 ...  0.          0.
  0.        ]
[ 0.38722098  0.39311126 -0.60365182 ...  0.          0.
  0.        ]
[ 0.39312929  0.38706937 -0.59564769 ...  0.          0.
  0.        ]
[ 0.39456123  0.38375023 -0.5165416  ...  0.          0.
  0.        ]
[ 0.39834911  0.37852523 -0.47689906 ...  0.          0.
  0.        ]
[ 0.40032211  0.37407854 -0.46413183 ...  0.          0.
  0.        ]
[ 0.41414413  0.37557968 -0.47190362 ...  0.          0.
  0.        ]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[ 0.42472157  0.37611628 -0.46268183 ...  0.          0.
  0.        ]
[ 0.42995757  0.37782

: 